In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
from awsglue.dynamicframe import DynamicFrame
import re

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1,None,pyspark,idle,,,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def sparkSqlQuery(glueContext, query, mapping, transformation_ctx) -> DynamicFrame:
    for alias, frame in mapping.items():
        frame.toDF().createOrReplaceTempView(alias)
    result = spark.sql(query)
    return DynamicFrame.fromDF(result, glueContext, transformation_ctx)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Simula los argumentos como si los estuvieras pasando por CLI
sys.argv = [
    'glue_script.py',  # nombre ficticio del script, solo para cumplir con la forma
    '--JOB_NAME', 'job_local_test',
    '--modo', 'test',
    '--meses', '6'
]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Obtener argumentos del job
args = getResolvedOptions(sys.argv, ['JOB_NAME', 'modo', 'meses'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# Inicializar GlueContext y Job
sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

logger = glueContext.get_logger()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def select_table_from_rd(table):
    node = glueContext.create_dynamic_frame.from_options(
    connection_type="redshift",
    connection_options={
        "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
        "useConnectionProperties": "true",
        "dbtable": table,
        "connectionName": "conexion-redshift"
    },
    transformation_ctx="node"
    )
    return node

def select_query_from_rd(query):
    node = glueContext.create_dynamic_frame.from_options(
        connection_type="redshift",
        connection_options={
            "sampleQuery": query,
            "redshiftTmpDir": "s3://aws-glue-assets-575238426438-us-east-1/temporary/",
            "useConnectionProperties": "true",
            "connectionName": "conexion-redshift"
        },
        transformation_ctx="node"
    )
    return node


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
node = select_table_from_rd("stdmx.stg_bodegas")
node.toDF().show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------------------+--------------------+
|bod_codigo|   bod_descripcion|        fecha_cargue|
+----------+------------------+--------------------+
|        39|BODEGA GUADALAJARA|2025-04-19 15:31:...|
|        80|       BODEGA MABE|2025-04-19 15:31:...|
|        88|BODEGA LOS ANGELES|2025-04-19 15:31:...|
|        89|   BODEGA DE CARPA|2025-04-19 15:31:...|
|        51|  BODEGA MS MEXICO|2025-04-19 15:31:...|
+----------+------------------+--------------------+
only showing top 5 rows

In [ ]:
query = "select * from gold.gi_unidad_tiempo where unidad_ano = 2025 and unidad_mes = 3"
node2 = select_query_from_rd(query)
node2.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{"unidad_id": 20250327.0000000000, "unidad_fecha": 2025-03-27 00:00:00.0, "unidad_num_dia_semana": null, "unidad_num_dia_mes": 27.0000000000, "unidad_num_dia_ano": null, "unidad_nombre_dia": null, "unidad_nombre_dia_abrev": null, "unidad_flag_dia_semana": null, "unidad_num_semana_ano": null, "unidad_num_semana_general": null, "unidad_fecha_inicio_semana": null, "unidad_id_inicio_semana": null, "unidad_mes": 3.0000000000, "unidad_nombre_mes": null, "unidad_nombre_mes_abrev": null, "unidad_trimestre": null, "unidad_ano": 2025.0000000000, "unidad_ano_mes": null, "unidad_mes_fiscal": null, "unidad_trimestre_fiscal": null, "unidad_ano_fiscal": null, "unidad_flag_ultimo_dia_seamana": null, "unidad_fecha_actualiz": 2025-01-01 00:00:00.0, "unidad_estado_actualiz": "A"}
{"unidad_id": 20250324.0000000000, "unidad_fecha": 2025-03-24 00:00:00.0, "unidad_num_dia_semana": null, "unidad_num_dia_mes": 24.0000000000, "unidad_num_dia_ano": null, "unidad_nombre_dia": null, "unidad_nombre_dia_abrev": null

In [ ]:
# Finalizar el trabajo
job.commit()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…